In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2, VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os


# Define paths
dataset_path = "/content/images/FREEDOM_FIGHHTERS"

# Data augmentation and loading
datagen = ImageDataGenerator(
    rescale=1.0/255,
    validation_split=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'

)



In [ ]:
train_generator.class_indices

In [ ]:
pip install tensorflow keras_facenet

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras_facenet import FaceNet

# Load pre-trained FaceNet model
facenet = FaceNet()
base_model = facenet.model

# Add custom layers
x = base_model.output
# x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)

# Define model
model_facenet = Model(inputs=base_model.input, outputs=predictions)

# Freeze base layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model_facenet.compile(optimizer=Adam(learning_rate=0.001),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_facenet_model.keras', save_best_only=True, monitor='val_accuracy')

# Train the model
history = model_facenet.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=100,
    callbacks=[early_stopping, model_checkpoint]
)

# Fine-tuning (optional)
# Unfreeze some layers for fine-tuning
for layer in model_facenet.layers[-20:]:
    layer.trainable = True

# Recompile the model
model_facenet.compile(optimizer=Adam(learning_rate=0.0001),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

# Train the model again (fine-tuning)
history_fine_tuning = model_facenet.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=50,
    callbacks=[early_stopping, model_checkpoint]
)

## Code with mtcnn to predict

In [5]:
! pip install tensorflow mtcnn opencv-python-headless numpy

   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB 325.1 kB/s eta 0:02:00
   ---------------------------------------- 0.0/38.8 MB 393.8 kB/s eta 0:01:39
   ---------------------------------------- 0.1/38.8 MB 722.1 kB/s eta 0:00:54
   ---------------------------------------- 0.2/38.8 MB 1.0 MB/s eta 0:00:38
   ---------------------------------------- 0.4/38.8 MB 1.7 MB/s eta 0:00:23
    --------------------------------------- 0.6/38.8 MB 1.9 MB/s eta 0:00:21
    --------------------------------------- 0.8/38.8 MB 2.3 MB/s eta 0:00:17
   - -------------------------------------- 1.0/38.8 MB 2.6 MB/s eta 0:00:15
   - -------------------------------------- 1.3/38.8 MB 2.9 MB/s eta 0:00:13
   - -------------------------------------- 1.5/38.8 MB 3.0 MB/s eta 0:00:13
   - -------------------------------------- 1.7/38.8 MB 3.2 MB/s eta 0:00:12
   -- -

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\vrajc\\miniconda3\\envs\\py312\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.



In [8]:
import tensorflow as tf
from mtcnn import MTCNN
from tensorflow.keras.preprocessing import image
import cv2
import numpy as np
from tensorflow.keras.saving import register_keras_serializable

@register_keras_serializable()
def scaling(x):
    return x / 255.0

@register_keras_serializable()
def l2_normalize(x, axis=-1, epsilon=1e-10):
    return x / np.sqrt(np.maximum(np.sum(np.square(x), axis=axis, keepdims=True), epsilon))

detector = MTCNN()

# Load the pre-trained model (replace with your actual model file)
model = tf.keras.models.load_model('./Model/facenet_87img_profile_face1.h5', custom_objects={'scaling': scaling, 'output_shape': (None, 88)})

# Class labels (replace with your actual class labels)
class_labels = [
    "A. P. J. Abdul Kalam", "Abhinav Bindra", "Adi Shankara", "Amrita Sher-Gil", 
    "Anandamayi Ma", "Ardeshir Godrej", "Atal Bihari Vajpayee", "Azim Premji", 
    "B. R. Ambedkar", "Bal Gangadhar Tilak", "Bankim Chandra Chattopadhyay", 
    "Basavanna", "Behramji Malabari", "Bhagat Singh", "Bipin Chandra Pal", 
    "Brijmohan Lall Munjal", "C. V. Raman", "Chandra Shekhar Azad", "Charan Singh", 
    "Dadabhai Naoroji", "Dhirubhai Ambani", "Dhondo Keshav Karve", "Dhyan Chand", 
    "G. D. Birla", "Gopal Ganesh Agarkar", "Guru Nanak", "Har Gobind Khorana", 
    "Homi J. Bhabha", "Indira Gandhi", "Ishwar Chandra Vidyasagar", "Ismat Chughtai", 
    "Jamini Roy", "Jamsetji Tata", "Jawaharlal Nehru", "Jiddu Krishnamurti", 
    "Jyotirao Phule", "Kabir", "Kapil Dev", "Karsanbhai Patel", "Khudiram Bose", 
    "Lakshmi Mittal", "Lal Bahadur Shastri", "Lala Lajpat Rai", "M. F. Husain", 
    "M. S. Swaminathan", "Madhva", "Mahatma Gandhi", "Mangal Pandey", "Mary Kom", 
    "Meghnad Saha", "Meher Baba", "Milkha Singh", "Morarji Desai", "Munshi Premchand", 
    "Nisargadatta Maharaj", "P. T. Usha", "P. V. Narasimha Rao", "Pandita Ramabai", 
    "Paramahansa Yogananda", "Prakash Padukone", "Rabindranath Tagore", 
    "Raja Ram Mohan Roy", "Rajendra Prasad", "Ramakrishna Paramahamsa", 
    "Ramana Maharshi", "Ramanuja", "Rani of Jhansi", "Sai Baba of Shirdi", 
    "Saina Nehwal", "Sardar Vallabhbhai Patel", "Sarojini Naidu", 
    "Sarvepalli Radhakrishnan", "Satyajit Ray", "Satyendra Nath Bose", 
    "Savitribai Phule", "Subhas Chandra Bose", "Sunil Gavaskar", 
    "Swami Dayananda Saraswati", "Swami Vivekananda", "Tarabai Shinde", 
    "Tulsi Tanti", "Tyeb Mehta", "Uday Shankar", "Udham Singh", "Verghese Kurien", 
    "Vikram Sarabhai", "Vinayak Damodar Savarkar", "Vishwanathan Anand"
]


# Function to detect and crop faces using MTCNN
def detect_and_crop_face_mtcnn(image_path, output_size=(224, 224)):
    # Read the image using OpenCV
    img = cv2.imread(image_path)
    if img is None:
        print(f"Image {image_path} not found!")
        return None

    # Convert the image from BGR (OpenCV format) to RGB (MTCNN works on RGB)
    rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Detect faces in the image
    faces = detector.detect_faces(rgb_img)

    # If faces are detected, crop the first detected face
    if len(faces) > 0:
        x, y, width, height = faces[0]['box']  # Coordinates of the first face
        face = img[y:y+height, x:x+width]  # Crop the face region
        # Resize the face to the desired output size
        face_resized = cv2.resize(face, output_size)
        return face_resized
    else:
        return None

# Function to preprocess the image and predict the class
def predict_image(image_path, model, target_size=(224, 224)):
    # First, detect and crop the face using MTCNN
    cropped_face = detect_and_crop_face_mtcnn(image_path, output_size=target_size)

    if cropped_face is None:
        print("No face detected!")
        return None

    # Convert the cropped face to RGB format (for Keras)
    cropped_face_rgb = cv2.cvtColor(cropped_face, cv2.COLOR_BGR2RGB)

    # Convert the cropped face to a numpy array and expand dimensions to match model input format
    face_array = np.array(cropped_face_rgb)
    face_array = np.expand_dims(face_array, axis=0)

    # Normalize the face array (assuming model requires normalization)
    face_array = face_array / 255.0

    # Predict the class of the face using the pre-trained model
    predictions = model.predict(face_array)

    # Get the predicted class index
    predicted_class_index = np.argmax(predictions, axis=1)[0]

    # Get the corresponding class label
    predicted_class_label = class_labels[predicted_class_index]

    return predicted_class_label

# Test the prediction function on an image
image_path = 'FaceNet/testing/apj_test1.jpg'
predicted_label = predict_image(image_path, model)

if predicted_label:
    print(f'The predicted label for the image is: {predicted_label}')


NotImplementedError: Exception encountered when calling Lambda.call().

[1mWe could not automatically infer the shape of the Lambda's output. Please specify the `output_shape` argument for this Lambda layer.[0m

Arguments received by Lambda.call():
  • args=('<KerasTensor shape=(None, 512), dtype=float32, sparse=False, name=keras_tensor_5042>',)
  • kwargs={'mask': 'None'}

In [2]:
from tensorflow.keras.models import load_model
import numpy as np
from sklearn.metrics import classification_report

# Step 1: Load the saved model
model_facenet = load_model('./Model/facenet_87img_profile_face1.h5')

# Step 2: Make predictions on the validation data
y_pred = model_facenet.predict(validation_generator)

# Step 3: Convert predictions to class labels
y_pred_classes = np.argmax(y_pred, axis=1)

# Get the true labels from the validation data
y_true = validation_generator.classes

# Step 4: Calculate and print the precision, recall, and F1-Score
print(classification_report(y_true, y_pred_classes, target_names=validation_generator.class_indices.keys()))


TypeError: Could not locate function 'scaling'. Make sure custom classes are decorated with `@keras.saving.register_keras_serializable()`. Full object config: {'module': 'builtins', 'class_name': 'function', 'config': 'scaling', 'registered_name': 'function'}

: 